Practice Cell to explore and validate the Bronze data - Mini Challenges 

In [4]:
from pathlib import Path
from pyspark.sql import functions as F
import os
import sys

current_dir = os.path.abspath('')
# Add parent directory to Python path to allow module imports
sys.path.append(os.path.abspath(os.path.join(current_dir,'..','..')))

from etl.common import get_spark, load_yml, project_root

root = project_root()
cfg = load_yml(str(root / "configs" / "bronze_config.yml"))
fmt = cfg["storage"]["format"]

spark = get_spark("bronze-mini-challenges")

bronze_matches_path = root / cfg["tables"]["matches"]["target_path"]
bronze_deliveries_path = root / cfg["tables"]["deliveries"]["target_path"]

matches = spark.read.format(fmt).load(str(bronze_matches_path))
deliveries = spark.read.format(fmt).load(str(bronze_deliveries_path))

Challenge 1 — Deliveries count vs matches

Goal: compute deliveries per match and see top matches (useful to spot super overs / anomalies).

In [6]:
dpm = ( deliveries
       .groupBy("match_id")
       .count()
       .withColumnRenamed("count","deliveries_count")
       )
print("Top 3 matches by deliveries_count")
(dpm.orderBy(F.desc("deliveries_count"))
 .limit(3)
 .show(truncate=False))

Top 3 matches by deliveries_count
+----------------+----------------+
|match_id        |deliveries_count|
+----------------+----------------+
|eef1376b596f046c|269             |
|755403f168d4e0fe|267             |
|af039871ce7fb9ae|265             |
+----------------+----------------+

